<a target="_blank"  href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/00-Setup/00-Setup_Redis_Workshops.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Redis Workshop

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

This notebook, as most of the notebooks in this series are designed to run either with build-in RedisStack or Redis Cloud.

To provision free forever instance of Redis Cloud:
- Head to https://redis.io/try-free/
- Register with email/gmail
- Create an **Essentials** subscription with 30MB free tier (no credit card required)
- Create RedisStack Database

In [1]:
# Install the requirements
!pip install -q redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 4.7 MB/s eta 0:00:00


### Install Redis Stack locally (OPTIONAL)

**NOTE: This step is OPTIONAL and commented out by default.**

We're now using Redis Cloud by default for persistent data storage across all workshops.

Uncomment the cell below only if you want to use a local Redis instance instead of Redis Cloud.

In [2]:
# COMMENTED OUT - Using Redis Cloud by default
# Uncomment the lines below if you want to use local Redis instead

# %%sh
# curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
# echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
# sudo apt-get update  > /dev/null 2>&1
# sudo apt-get install redis-stack-server  > /dev/null 2>&1
# redis-stack-server --daemonize yes

print("✅ Using Redis Cloud - no local installation needed!")


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Setup Redis Cloud Connection
**Using Redis Cloud by default for persistent data storage across all workshops.**

This provides:
- ✅ Persistent data across sessions
- ✅ Shared data between workshops
- ✅ No local Redis setup required
- ✅ 250MB storage capacity

In [3]:
import os

# Redis Cloud Configuration (Default)
# This Redis Cloud instance provides persistent storage for all workshops
REDIS_HOST = os.getenv("REDIS_HOST", "redis-15306.c329.us-east4-1.gce.redns.redis-cloud.com")
REDIS_PORT = os.getenv("REDIS_PORT", "15306")
REDIS_USERNAME = os.getenv("REDIS_USERNAME", "default")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "ftswOcgMB8dLCnjbKMMDBg3DNnpi1KO5")

# For local Redis (uncomment if needed):
# REDIS_HOST = "localhost"
# REDIS_PORT = "6379"
# REDIS_PASSWORD = ""

# Shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# Redis URL for libraries that need it
REDIS_URL = f"redis://{REDIS_USERNAME}:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

print(f"🔗 Connecting to Redis Cloud: {REDIS_HOST}:{REDIS_PORT}")


### Basic Test with Redis CLI

In [4]:
!redis-cli $REDIS_CONN PING
!redis-cli $REDIS_CONN SET hello world
!redis-cli $REDIS_CONN GET hello

PONG
OK
"world"


### Basic Test with Python client

In [5]:
import redis

r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD
)

r.get("hello")

b'world'

### TTL - Time To Live

It's extremely important for caching systems to control for how long values will be stored. Redis allows you to control expiry with the EXPIRE command and with flags like EX, EXAT etc. For more information see https://redis.io/commands/expire/ and https://redis.io/commands/set/

In [6]:
import time

r.set(name="ttl_key", value="this key will expire soon", ex=10)

for i in range(15):
  print(r.get(name="ttl_key"))
  time.sleep(1)

b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
None
None
None
None
None


#### Cleanup
(Optional) Delete all keys and indexes from the database.

In [7]:
###-- FLUSHDB will wipe out the entire database!!! Use with caution --###
# !redis-cli $REDIS_CONN flushdb